# Import Modules

In [1]:
import pandas as pd
import numpy as np
import collections
import seaborn as sns
import os
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
from sklearn import svm

# Store Model Information

In [2]:
model_cols = ['ModelName','Accuracy','Precision','Recall','Mean Squared Error']
model_info = pd.DataFrame(columns=model_cols)

# Load Cleaned Data

In [3]:
filePath = os.path.join('../','../','../','Data','Stage','NoShowAppointments_cleaned.csv')
df = pd.read_csv(filePath,low_memory=False)

# Build data to model

In [4]:
X = df.drop(['NoShow','AppointmentID','PatientId'], axis=1)
y = df['NoShow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# TensorFlow

In [5]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [6]:
X = df.drop(['NoShow','AppointmentID','PatientId'],axis=1).values
y = df['NoShow'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=101)

In [7]:
scaler = MinMaxScaler()

In [8]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [9]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
model = Sequential()

# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw

model.add(Dense(units=110,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=50,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=25,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,activation='sigmoid'))

# For a binary classification problem
model.compile(loss='binary_crossentropy', optimizer='adam')

In [11]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [ ]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop]
          )

Train on 82894 samples, validate on 27632 samples
Epoch 1/600
82894/82894 [==============================] - 6s 74us/sample - loss: 0.4258 - val_loss: 0.2728
Epoch 2/600
82894/82894 [==============================] - 4s 53us/sample - loss: 0.2698 - val_loss: 0.2367
Epoch 3/600
82894/82894 [==============================] - 5s 55us/sample - loss: 0.2472 - val_loss: 0.2362
Epoch 4/600
82894/82894 [==============================] - 4s 53us/sample - loss: 0.2441 - val_loss: 0.2343
Epoch 5/600
82894/82894 [==============================] - 4s 53us/sample - loss: 0.2424 - val_loss: 0.2328
Epoch 6/600
82894/82894 [==============================] - 4s 53us/sample - loss: 0.2407 - val_loss: 0.2335
Epoch 7/600
82894/82894 [==============================] - 5s 55us/sample - loss: 0.2395 - val_loss: 0.2326
Epoch 8/600
82894/82894 [==============================] - 4s 53us/sample - loss: 0.2382 - val_loss: 0.2323
Epoch 9/600
82894/82894 [==============================] - 4s 52us/sample - loss: 0.23

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

In [ ]:
predictions = model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, predictions))
print("Precision:",metrics.precision_score(y_test, predictions))
print("Recall:",metrics.recall_score(y_test, predictions))

In [ ]:
name = 'Artificial Neural Network (ANN)'
accuracy = metrics.accuracy_score(y_test, predictions)
precision = metrics.precision_score(y_test, predictions)
recall = metrics.recall_score(y_test, predictions)
msqe = metrics.mean_squared_error(y_test, predictions)
model_info = model_info.append(
    {
        'ModelName': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'Mean Squared Error': msqe        
    },
    ignore_index=True)

In [ ]:
#save model
model.save('final_apptnoshow_model.h5')

# Show Table Metrics

In [ ]:
model_info

# Save Scaler

In [ ]:
import joblib

In [ ]:
joblib.dump(scaler,'apptnoshow_scaler.pkl')